In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
img_1 = [
    "predictions/rgb/image_8.png",
    "predictions/gt/gt_8.png",
    "predictions/pred/pred_8.png",
    None,
]
img_2 = [
    "predictions/rgb/image_32.png",
    "predictions/gt/gt_32.png",
    "predictions/pred/pred_32.png",
    None,
]
img_3 = [
    "predictions/rgb/image_14.png",
    "predictions/gt/gt_14.png",
    "predictions/pred/pred_14.png",
    None,
]
img_4 = [
    "predictions/rgb/image_28.png",
    "predictions/gt/gt_28.png",
    "predictions/pred/pred_28.png",
    None,
]
img_5 = [
    "predictions/rgb/image_13.png",
    "predictions/gt/gt_13.png",
    "predictions/pred/pred_13.png",
    None,
]

read_rgb = lambda x: np.array(Image.open(x).convert("RGB"))
read_gray = lambda x: np.array(Image.open(x).convert("L"))

data = [img_1, img_2, img_3, img_4, img_5]

In [ ]:
import numpy as np
import cv2
from PIL import Image, ImageDraw


# def create_overlay_image(original_img, ground_truth_mask, predicted_mask):
#     # Convert binary masks to uint8 format
#     ground_truth_mask = ground_truth_mask.astype(np.uint8) * 255
#     predicted_mask = predicted_mask.astype(np.uint8) * 255

#     # Create an RGB version of the original image
#     original_rgb = original_img

#     # Create an alpha channel for overlaying masks
#     alpha_channel = np.zeros_like(original_img)

#     # Set red for ground truth and blue for predicted mask in alpha channel
#     alpha_channel[ground_truth_mask > 0] = [255, 0, 0]
#     alpha_channel[predicted_mask > 0] = [0, 0, 255]

#     # Combine original image and masks using alpha blending
#     overlayed_img = cv2.addWeighted(original_rgb, 0.7, alpha_channel, 0.3, 0)

#     return overlayed_img

import numpy as np
import matplotlib.pyplot as plt
from skimage import measure, color

def create_overlay_image(image, ground_truth_mask, predicted_mask):
    """
    Overlays the boundaries of the ground truth mask and the predicted mask on the image.
    
    Parameters:
    - image: numpy array, the original image
    - ground_truth_mask: numpy array, binary ground truth mask
    - predicted_mask: numpy array, binary predicted mask
    
    Returns:
    - overlay_image: numpy array, the original image with overlaid boundaries
    """
    def generate_boundary(mask):
        # Ensure the mask is binary
        mask = mask.astype(bool)
        
        # Find contours at a constant value of 0.5
        contours = measure.find_contours(mask, 0.5)
        
        # Create an empty image to draw the boundaries
        boundary = np.zeros_like(mask, dtype=np.uint8)
        
        # Draw the contours on the boundary image
        for contour in contours:
            for y, x in contour:
                boundary[int(y), int(x)] = 1
                
        return boundary
    
    # Generate boundaries
    ground_truth_boundary = generate_boundary(ground_truth_mask)
    predicted_boundary = generate_boundary(predicted_mask)
    
    # Convert the original image to RGB if it's grayscale
    if len(image.shape) == 2 or image.shape[2] == 1:
        image = color.gray2rgb(image)
    
    # Create a copy of the image to overlay the boundaries
    overlay_image = image.copy()
    
    # Define colors for boundaries
    ground_truth_color = [255, 0, 0]  # Red
    predicted_color = [0, 0, 255]     # Blue
    
    # Overlay the ground truth boundary
    overlay_image[ground_truth_boundary == 1] = ground_truth_color
    
    # Overlay the predicted boundary
    overlay_image[predicted_boundary == 1] = predicted_color
    
    return overlay_image


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MultipleLocator
from PIL import Image

# Assuming 'data' is your list of lists
# Each inner list should contain [image, ground truth, prediction]

# Create a grid layout
rows = 4
cols = len(data)
fig = plt.figure(figsize=(cols * 2, rows * 2))
gs = gridspec.GridSpec(rows, cols, wspace=0.01, hspace=0.01)

# Plot each image, ground truth, and prediction
for col, sample in enumerate(data):
    for row, img_type in enumerate(sample):

        if row != 3:
            ax = plt.subplot(gs[row, col])
            img = Image.open(img_type)

        if row == 3:
            img = create_overlay_image(
                np.array(Image.open(sample[0])),
                np.array(Image.open(sample[1]).convert("L")),
                np.array(Image.open(sample[2]).convert("L")),
            )

        # Display image
        ax.imshow(img, cmap="gray")  # Assuming images are in grayscale
        ax.axis("off")

        # Set titles on the leftmost side
        if col == 0 and row != 3:
            ax.text(
                -0.01,
                0.5,
                f"({chr(97+row)})",
                transform=ax.transAxes,
                fontsize=12,
                va="center",
                ha="right",
            )

        # # Set titles for each row
        # if row == 0:
        #     ax.set_title('Image')
        # elif row == 1:
        #     ax.set_title('Ground Truth')
        # elif row == 2:
        #     ax.set_title('Prediction')

# Adjust layout
plt.tight_layout()
plt.savefig("combined_2016_new.png", bbox_inches="tight")
plt.show()

In [ ]:
# def plot_images(input_list):
#     num_images = len(input_list)

#     # Set up the subplots
#     fig, axes = plt.subplots(3, num_images, squeeze=True)

#     # Set titles on the left side
#     titles = ["Image", "Mask", "Prediction"]
#     for i, title in enumerate(titles):
#         fig.text(
#             0.03,
#             0.5,
#             f"{title}",
#             va="center",
#             ha="center",
#             rotation="vertical",
#             fontsize=9,
#         )

#     for i, input_paths in enumerate(input_list):
#         img_path, mask_path, pred_path = input_paths

#         # Load images
#         img = mpimg.imread(img_path)
#         mask = mpimg.imread(mask_path)
#         pred = mpimg.imread(pred_path)

#         axes[2, i].set_aspect("equal")

#         # Plot the images
#         axes[0, i].imshow(img)
#         axes[1, i].imshow(mask, cmap="gray")
#         axes[2, i].imshow(pred, cmap="gray")

#         # Turn off axis labels
#         for ax in axes[:, i]:
#             ax.axis("off")

#     plt.tight_layout(h_pad=0, w_pad=0, pad=0.0001)
#     plt.subplots_adjust(wspace=0.01, hspace=0.01)
#     plt.savefig("png_2016.png")
#     plt.show()

In [ ]:
# plot_images(data)

In [ ]:
# from PIL import Image, ImageDraw
# import math


# def combine_images_with_masks(image_data, output_path):
#     num_images = len(image_data)
#     cols = num_images  # Number of columns in the output image grid
#     rows = 3  # Number of rows (images, masks, overlays)

#     # Create a blank canvas for the combined image
#     combined_width = cols * 200  # Adjust the width of each image as needed
#     combined_height = rows * 200  # Adjust the height of each image as needed
#     combined_image = Image.new("RGB", (combined_width, combined_height), color="white")

#     # Paste each image, mask, and overlay onto the canvas in separate rows
#     for i in range(num_images):
#         image_path, mask_path, overlay_path = image_data[i]

#         # Load image, mask, and overlay
#         img = Image.open(image_path)
#         mask = Image.open(mask_path).convert("L")  # Convert to grayscale if needed
#         overlay = Image.open(overlay_path).convert("L")

#         # Resize each image, mask, and overlay as needed
#         img = img.resize((200, 200))
#         mask = mask.resize((200, 200))
#         overlay = overlay.resize((200, 200))

#         # Paste image onto the canvas (row 1)
#         x = i * 200
#         y = 0
#         combined_image.paste(img, (x, y))

#         # Paste mask onto the canvas (row 2)
#         y = 200
#         combined_image.paste(mask, (x, y))

#         # Paste overlay onto the canvas (row 3)
#         y = 400
#         combined_image.paste(overlay, (x, y))

#     # Save or display the combined image
#     combined_image.save(output_path)
#     combined_image.show()


# output_image_path = "combined_2016.jpg"
# combine_images_with_masks(data, output_image_path)